### 기본 세팅

In [1]:
import pandas as pd
import ast
import json
import urllib3

from tqdm.notebook import tqdm

### 데이터 로드

In [2]:
counseling_전체_df    = pd.read_csv('counseling_전체.csv', index_col=0)
voicephishing_전체_df = pd.read_csv('voicephishing_전체.csv', index_col=0)
counseling_전체_df['보이스피싱 여부']    = 0
voicephishing_전체_df['보이스피싱 여부'] = 1

textdata_전체_df = pd.concat([counseling_전체_df, voicephishing_전체_df], ignore_index=True)
textdata_전체_df

,내용,보이스피싱 여부
0,"['네, 안녕하세요.', '오늘 제가 연락드린 건 제가 지금 대출을 받아서 갚고 있...",0
1,"['문의하고 싶은 게 있어서요.', '담보대출 관련 문의 좀 드릴게요.', '죄송한...",0
2,"['안녕하세요.', '국민은행 상담사 김지원입니다.', '네, 안녕하세요.', '무...",0
3,"['안녕하세요.', '미스 은행 상담원 은가은입니다.', '무엇을 도와드릴까요?',...",0
4,"['네 안녕하세요.', '네 제가 선물옵션 투자를 좀 하고 싶어서 전화드렸습니다.'...",0
...,...,...
336,"['여보세요', '여보세요', 'OOO씨 혹시 맞습니까?', '네', '저는 서울 ...",1
337,"['여보세요', '네', '네 안녕하십니까.', '혹시 OOO 핸드폰 맞으십니까?'...",1
338,"['수고하십니다.', '여기는 서울 중앙 지검이고요', '네', '저는 1,000만...",1
339,"['어 이름은 김호철이고 태어난 곳은 전라도 광주, 올해로 사십세된 남성에 농협 은...",1


In [3]:
# 문자열의 리스트화
def str2list(textdata):
    textdata = [ast.literal_eval(case) for case in textdata['내용']]
    return textdata


# 문장 마지막에 문장부호가 없는 경우 온점 추가
def add_sentence_spots(textdata):
    for i in range(len(textdata)):
        # 사건 단위로 진행
        cur_case = textdata[i]
        
        for j in range(len(cur_case)):
            # 사건 안의 문장 단위로 진행
            cur_sent = cur_case[j]
            len_cur_sent = len(cur_sent)
            
            # 문장 내용이 없는 경우 제거
            if cur_sent == '':
                textdata[i].remove(cur_sent)
                continue
            
            # 문장부호가 없는 경우 온점 추가
            if (cur_sent[len_cur_sent-1] != '.') and (cur_sent[len_cur_sent-1] != '?'):
                cur_sent = cur_sent + '.'
                textdata[i][j] = cur_sent
            else:
                pass               
    return textdata


# 텍스트 길이가 10000 이상인 사건은 그 뒷문장 제거
def adjusting_sent_length(textdata):
    # 글자수 확인을 위해 일단 합침
    cases = [' '.join(sent for sent in case) for case in textdata]
    case_length = [len(case) for case in cases]
    for i in range(len(textdata)):
        if case_length[i] >= 10000:
            textdata[i] = reduce_length(textdata[i])
    return textdata


# 사건 내 긴 문장들 제거
def reduce_length(case):
    out_case = []
    total_length = 0
    for sent in case:
        if (total_length + (len(sent)+1)) < 10000: 
            out_case.append(sent)
            total_length += (len(sent)+1)
        else:
            return out_case
        
        
# 사건마다 문장을 모두 합침
def concat_sent_incase(textdata):
    textdata = [' '.join(sent for sent in case) for case in textdata]
    return textdata

In [4]:
textdata_전체 = str2list(textdata_전체_df)
textdata_전체 = add_sentence_spots(textdata_전체)
textdata_전체 = adjusting_sent_length(textdata_전체)
textdata_전체 = concat_sent_incase(textdata_전체)

### ETRI NER 태깅

In [5]:
class WordTagger:
    def __init__(self, key):
        self.openApiURL     = "http://aiopen.etri.re.kr:8000/WiseNLU_spoken"
        self.accessKey      = key
        self.analysisCode   = "ner"
        self.sentences_json = []
        self.tags = {'OGG_ECONOMY': '경제기관',
                     'OGG_POLITICS': '공공기관',
                     'CV_POSITION': '직위',
                     'CV_RELATION': '인간관계',
                     'CV_OCCUPATION': '직업',
                     'OGG_LAW':'법률기관'
                     'OGG_MEDIA':'미디어기관'
                     'QT_PRICE': '금액',
                     'DT_DURATION': '기간',
                     'TMI_HW': '하드웨어',
                     'TMI_SERVICE': 'IT서비스',
                     'TI_DURATION': '기간',
                     'CV_LAW': '법률'
        }
        
        
    # 사건별 NER 분석 결과를 sentences_json에 저장
    def generate_NERInfo(self, textdata):
        self.sentences_json = []
        for case in tqdm(textdata):
            result = self.requst_text_to_NER_json(case)
            self.sentences_json.append(result)
        
        
    # ETRI 요청으로 텍스트의 문장정보 json 결과 반환
    def requst_text_to_NER_json(self, text):
        requestJson = {
            "access_key": self.accessKey,
            "argument": {
                "text": text,
                "analysis_code": self.analysisCode
            }
        }
        
        try:
            http     = urllib3.PoolManager()
            response = http.request("POST",
                self.openApiURL,
                headers={"Content-Type": "application/json; charset=UTF-8"},
                body=json.dumps(requestJson)
            )
            if response.status == 200:
                result = json.loads(response.data)
                return result['return_object']['sentence']
            else:
                return []
        except:
            return []
        
    
    # 생성된 NERInfo로 태깅된 문장 반환
    def word_tagging(self, convert=True, ext=False, kor=True):
        """
            convert = True: 문장에 NER 태깅 변환 (기본값)
            convert = False: 문장에 NER 태깅 없이 그대로 진행
            ext = False: 중요 태그 상위 5% 적용 (기본값)
            ext = True: 중요 태그 상위 10% 적용
            kor = True: 태그명 한글로 변환 (기본값)
            kor = False: 태그명 영어로 변환
        """
        # 중요 태그 상위 5% 적용
        target_tags = ['OGG_ECONOMY', 'OGG_POLITICS', 'CV_POSITION', 'CV_RELATION', 'CV_OCCUPATION', 'QT_PRICE','OGG_LAW']
        # 중요 태그 상위 10% 적용
        if ext == True:
            target_tags += ['DT_DURATION', 'TMI_HW', 'TMI_SERVICE','OGG_MEDIA', 'TI_DURATION', 'CV_LAW']
        
        out_list = []             # 전체 사건 리스트
        for case in self.sentences_json:     
            new_sentence = []     # 사건당 문장 리스트
            
            # NER 태깅을 안하는 경우
            if convert == False: 
                new_sentence = [sent['text'].strip() for sent in case]
            # NER 태깅을 하는 경우
            else:
                for sent in case:     
                    ner_list = [] 
                    for item in sent['NE']:
                        # 중요 태그에 포함될 경우
                        if item['type'] in target_tags:                      
                            ner_list.append((item['text'], item['type']))     # 각 문장에서 (단어,태그) 형태로 수집
                    new_sent = self.replace_word_to_nertag(sent['text'], ner_list, kor)
                    new_sentence.append(new_sent)
            out_list.append(new_sentence)
        return out_list               
    
    
    # 문장에서 해당되는 단어를 태그로 변경
    def replace_word_to_nertag(self, text, nerlist, kor):
        """
            text: 원래 문장
            nerlist: (단어, 태그)로 구성된 NER 리스트
            kor: 한글 변환 여부
        """
        out_text = text
        for word, tag in nerlist:
            if kor == True:
                tag_token = '<%s>' % self.tags[tag]
            else:
                tag_token = '<%s>' % tag
            out_text = out_text.replace(word, tag_token)
        return out_text.strip()   

In [6]:
# ETRI 키로 NER 변환 객체 생성
ner_tagger = WordTagger(key="") #ETRI 키 입력

# ETRI 요청으로 json 정보 수집
ner_tagger.generate_NERInfo(textdata_전체)

  0%|          | 0/341 [00:00<?, ?it/s]

### 조건별 데이터 변환

In [7]:
# 변환 데이터 생성 1 - 원문
original    = ner_tagger.word_tagging(convert=False)

df_original = textdata_전체_df
df_original['내용'] = original
df_original.to_csv('./path/입력할파일', encoding='utf-8', index=False)

In [8]:
df_original

,내용,보이스피싱 여부
0,"[네, 안녕하세요., 오늘 제가 연락드린 건 제가 지금 대출을 받아서 갚고 있는 중...",0
1,"[문의하고 싶은 게 있어서요., 담보대출 관련 문의 좀 드릴게요., 죄송한데 제가 ...",0
2,"[안녕하세요., 국민은행 상담사 김지원입니다., 네, 안녕하세요., 무엇을 도와드릴...",0
3,"[안녕하세요., 미스 은행 상담원 은가은입니다., 무엇을 도와드릴까요?, 네, 안녕...",0
4,"[네 안녕하세요., 네 제가 선물옵션 투자를 좀 하고 싶어서 전화드렸습니다., 제가...",0
...,...,...
336,"[여보세요., 여보세요., OOO씨 혹시 맞습니까?, 네., 저는 서울 중앙 지방 ...",1
337,"[여보세요., 네., 네 안녕하십니까., 혹시 OOO 핸드폰 맞으십니까?, 어디세요...",1
338,"[수고하십니다., 여기는 서울 중앙 지검이고요., 네., 저는 1,000만 범죄 수...",1
339,"[어 이름은 김호철이고 태어난 곳은 전라도 광주, 올해로 사십세된 남성에 농협 은행...",1


In [9]:
# 변환 데이터 생성 2 - 한글, 중요 태그 5%
kor_ratio5    = ner_tagger.word_tagging()

df_kor_ratio5 = textdata_전체_df
df_kor_ratio5['내용'] = kor_ratio5
df_kor_ratio5.to_csv('./path/입력할파일', encoding='utf-8', index=False)

In [10]:
df_kor_ratio5

,내용,보이스피싱 여부
0,"[네, 안녕하세요., 오늘 제가 연락드린 건 제가 지금 대출을 받아서 갚고 있는 중...",0
1,"[문의하고 싶은 게 있어서요., 담보대출 관련 문의 좀 드릴게요., 죄송한데 제가 ...",0
2,"[안녕하세요., <경제기관> <직업> 김지원입니다., 네, 안녕하세요., 무엇을 도...",0
3,"[안녕하세요., <경제기관> <직위> 은가은입니다., 무엇을 도와드릴까요?, 네, ...",0
4,"[네 안녕하세요., 네 제가 선물옵션 투자를 좀 하고 싶어서 전화드렸습니다., 제가...",0
...,...,...
336,"[여보세요., 여보세요., OOO씨 혹시 맞습니까?, 네., 저는 서울 중앙 지방 ...",1
337,"[여보세요., 네., 네 안녕하십니까., 혹시 OOO 핸드폰 맞으십니까?, 어디세요...",1
338,"[수고하십니다., 여기는 <공공기관>이고요., 네., 저는 <금액> 범죄 수사 일 ...",1
339,"[어 이름은 김호철이고 태어난 곳은 전라도 광주, 올해로 사십세된 남성에 <경제기관...",1


In [11]:
# 변환 데이터 생성 3 - 한글, 중요 태그 10%
kor_ratio10    = ner_tagger.word_tagging(ext=True)

df_kor_ratio10 = textdata_전체_df
df_kor_ratio10['내용'] = kor_ratio10
df_kor_ratio10.to_csv('./path/입력할파일', encoding='utf-8', index=False)

In [12]:
df_kor_ratio10

,내용,보이스피싱 여부
0,"[네, 안녕하세요., 오늘 제가 연락드린 건 제가 지금 대출을 받아서 갚고 있는 중...",0
1,"[문의하고 싶은 게 있어서요., 담보대출 관련 문의 좀 드릴게요., 죄송한데 제가 ...",0
2,"[안녕하세요., <경제기관> <직업> 김지원입니다., 네, 안녕하세요., 무엇을 도...",0
3,"[안녕하세요., <경제기관> <직위> 은가은입니다., 무엇을 도와드릴까요?, 네, ...",0
4,"[네 안녕하세요., 네 제가 선물옵션 투자를 좀 하고 싶어서 <하드웨어>드렸습니다....",0
...,...,...
336,"[여보세요., 여보세요., OOO씨 혹시 맞습니까?, 네., 저는 서울 중앙 지방 ...",1
337,"[여보세요., 네., 네 안녕하십니까., 혹시 OOO <하드웨어> 맞으십니까?, 어...",1
338,"[수고하십니다., 여기는 <공공기관>이고요., 네., 저는 <금액> 범죄 수사 일 ...",1
339,"[어 이름은 김호철이고 태어난 곳은 전라도 광주, 올해로 사십세된 남성에 <경제기관...",1


In [13]:
# 변환 데이터 생성 4 - 영어, 중요 태그 5%
eng_ratio5    = ner_tagger.word_tagging(kor=False)

df_eng_ratio5 = textdata_전체_df
df_eng_ratio5['내용'] = eng_ratio5
df_eng_ratio5.to_csv('./path/입력할파일', encoding='utf-8', index=False)

In [14]:
df_eng_ratio5

,내용,보이스피싱 여부
0,"[네, 안녕하세요., 오늘 제가 연락드린 건 제가 지금 대출을 받아서 갚고 있는 중...",0
1,"[문의하고 싶은 게 있어서요., 담보대출 관련 문의 좀 드릴게요., 죄송한데 제가 ...",0
2,"[안녕하세요., <OGG_ECONOMY> <CV_OCCUPATION> 김지원입니다....",0
3,"[안녕하세요., <OGG_ECONOMY> <CV_POSITION> 은가은입니다., ...",0
4,"[네 안녕하세요., 네 제가 선물옵션 투자를 좀 하고 싶어서 전화드렸습니다., 제가...",0
...,...,...
336,"[여보세요., 여보세요., OOO씨 혹시 맞습니까?, 네., 저는 서울 중앙 지방 ...",1
337,"[여보세요., 네., 네 안녕하십니까., 혹시 OOO 핸드폰 맞으십니까?, 어디세요...",1
338,"[수고하십니다., 여기는 <OGG_POLITICS>이고요., 네., 저는 <QT_P...",1
339,"[어 이름은 김호철이고 태어난 곳은 전라도 광주, 올해로 사십세된 남성에 <OGG_...",1


In [15]:
# 변환 데이터 생성 5 - 영어, 중요 태그 10%
eng_ratio10    = ner_tagger.word_tagging(kor=False, ext=True)

df_eng_ratio10 = textdata_전체_df
df_eng_ratio10['내용'] = eng_ratio10
df_eng_ratio10.to_csv('./path/입력할파일', encoding='utf-8', index=False)

In [16]:
df_eng_ratio10

,내용,보이스피싱 여부
0,"[네, 안녕하세요., 오늘 제가 연락드린 건 제가 지금 대출을 받아서 갚고 있는 중...",0
1,"[문의하고 싶은 게 있어서요., 담보대출 관련 문의 좀 드릴게요., 죄송한데 제가 ...",0
2,"[안녕하세요., <OGG_ECONOMY> <CV_OCCUPATION> 김지원입니다....",0
3,"[안녕하세요., <OGG_ECONOMY> <CV_POSITION> 은가은입니다., ...",0
4,"[네 안녕하세요., 네 제가 선물옵션 투자를 좀 하고 싶어서 <TMI_HW>드렸습니...",0
...,...,...
336,"[여보세요., 여보세요., OOO씨 혹시 맞습니까?, 네., 저는 서울 중앙 지방 ...",1
337,"[여보세요., 네., 네 안녕하십니까., 혹시 OOO <TMI_HW> 맞으십니까?,...",1
338,"[수고하십니다., 여기는 <OGG_POLITICS>이고요., 네., 저는 <QT_P...",1
339,"[어 이름은 김호철이고 태어난 곳은 전라도 광주, 올해로 사십세된 남성에 <OGG_...",1
